In [ ]:

import numpy as np
from scipy.optimize import fsolve
from scipy.special import softmax
import pandas as pd
#from numpy import vstack
#from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score



def OSA(X):  #Optical Saturable Absorber, numerically returns Tm value given input intensity vector.
        T0 = 0.9
        Tm = X
        sigma = 10e-15
        tau = 10e-9
        c=0
        
        for i in X:
            neg_flag = 0
            if i < 0:
                i0 = -i
                neg_flag = 1
            elif i == 0:
                i0 = 0.1
            else:
                i0 = i

            func = lambda Tmp : i0*sigma*tau - 0.5*(np.log((Tmp/T0)))/(1.0 - Tmp) 
            Tm_guess = 0.1
            if neg_flag == 0:
                Tm[c] = fsolve(func, Tm_guess)
                c=c+1
            else:
                Tm[c] = -1 * fsolve(func, Tm_guess)
                c=c+1
        return Tm


#Forward Propagation


def forward(X,w1,w2):
    a1= w1.dot(X)
    z1=OSA(a1)
    a2=w2.dot(z1)
    z2=OSA(a2)
    h=softmax(abs(a2))
    return h, z1, z2, a1, a2

def dactivation(Tm, I):      #Given Tm and Intensity, returns dactivation for Backpropagation update algorithm
    sigma = 10e-15
    tau = 10e-9
    B = 2 * sigma * tau
    return (Tm * B * (1 - Tm))/(B * I * Tm + 1)

def backward(h,targets, input, z1, z2, a1, a2, dact1, dact2, w2):  #return backward propagation of errors
    dw2 = np.outer((h-targets) @ np.sign(h) * dact2, z1)
    dw1 = np.outer(((h-targets) @ np.sign(h) * dact2 @ w2) * dact1, input)
    return dw1 , dw2

def gradient(w1, w2, dw1, dw2, stepsize):
    nw1 = w1 - stepsize * dw1
    nw2 = w2 - stepsize * dw2
    return nw1, nw2

def encoder(target):
    t=np.zeros(11)
    t[target]= 1
    return t

def train_batch(X, w1, w2, batch_size, nhid, stepsize):  #train on a batch

    avg_w1 = np.zeros((nhid, 10))
    avg_w2 = np.zeros((11, nhid))
    
    for i in range(batch_size):
        [h, z1, z2, a1, a2] = forward(X[i,:-1], w1, w2)
        dact1 = dactivation(z1, a1)
        dact2 = dactivation(z2, a2)
        t = encoder(X[i,-1].astype(int))
        [dw1, dw2] = backward(h, t, X[i,:-1], z1, z2, a1, a2, dact1, dact2, w2)
        avg_w1=avg_w1+dw1
        avg_w2=avg_w2+dw2

    avg_w1=avg_w1/batch_size
    avg_w2=avg_w2/batch_size
    [nw1, nw2] = gradient(w1,w2,avg_w1,avg_w2, stepsize)
    return nw1, nw2

def train_model(nhid, batch_size, stepsize, epochs):

    #Initialization
    w1 = np.array(np.random.rand(nhid,10)-0.5)
    w2 = np.array(np.random.rand(11,nhid)-0.5)

    #Reading dataset from .csv files

    dataset=pd.read_csv('train.csv', sep=',',header=None)

    batch_per_epoch = 6

    input=np.array(dataset)
  

      
    for j in range(epochs):
        for b in range(batch_per_epoch):
            [w1, w2] = train_batch(input[(batch_size*b):(batch_size*(b+1)),:], w1, w2, batch_size, nhid, stepsize)
        

    return w1, w2


def validator(w1, w2):

    df=pd.read_csv('train.csv', sep=',',header=None)
    inp=np.array(df)

    correct = 0
    indx=np.zeros(528)
    for i in range(20):
       [hj, g1, g2, x1, x2] = forward(inp[i,:-1],w1,w2)
       
       
       indx[i]=np.argmax(abs(x2)) 
       print(inp[i,-1].astype(int))
       print(np.argmax(abs(x2)))
       if np.argmax(abs(x2)) == (inp[i,-1].astype(int)):
          correct = correct +1
           
    
    return correct/20

def predict(input, w1,w2):

    [h, z1, z2, a1, a2] = forward(input,w1,w2)
    
    return h


stepsize = 0.1
batch_size = 88
nhid = 3 #number of the neurons in the hidden layer
epochs = 80

[w1, w2] = train_model(nhid, batch_size, stepsize, epochs)

test=[-3.86, 2.11, -0.939,	0.688,	-0.675,	1.679,	-0.512,	0.928,	-0.167,	-0.434]



print(validator(w1,w2))